In [30]:
import pandas as pd


In [31]:
df = pd.read_excel("data_3\\信息\\单品批发价与损耗.xlsx")
df.head()

,单品名,成本,亏损比,销量最小值,销量最大值,成本加成定价最小值,成本加成定价最大值,销量约束最小值,销量约束最大值,成本加成定价约束最小值,成本加成定价约束最大值,补货量最小值
0,白玉菇(袋),4.364556,6.57,1.000,1.00,4.10,6.30,0.8000,1.200,3.280,7.560,2.675800
1,菠菜,9.660000,18.51,2.200,3.30,16.00,17.00,1.7600,3.960,12.800,20.400,3.067861
2,菠菜(份),4.070000,9.43,2.000,15.00,3.90,6.10,1.6000,18.000,3.120,7.320,2.760296
3,菜心,4.620000,13.70,0.822,4.12,6.58,6.61,0.6576,4.944,5.264,7.932,2.896871
4,虫草花(份),2.600000,9.43,1.000,4.00,4.50,4.70,0.8000,4.800,3.600,5.640,2.760296


In [32]:
df = pd.read_excel("data_3\\信息\\单品批发价与损耗.xlsx").drop(["销量最小值", "销量最大值", "成本加成定价最小值", "成本加成定价最大值"], axis=1)
name = df.iloc[:, 0]
df.head()

,单品名,成本,亏损比,销量约束最小值,销量约束最大值,成本加成定价约束最小值,成本加成定价约束最大值,补货量最小值
0,白玉菇(袋),4.364556,6.57,0.8000,1.200,3.280,7.560,2.675800
1,菠菜,9.660000,18.51,1.7600,3.960,12.800,20.400,3.067861
2,菠菜(份),4.070000,9.43,1.6000,18.000,3.120,7.320,2.760296
3,菜心,4.620000,13.70,0.6576,4.944,5.264,7.932,2.896871
4,虫草花(份),2.600000,9.43,0.8000,4.800,3.600,5.640,2.760296


Q = 成本加成定价($F_1$) * 补货量($N$) * (1-k) - 成本($F_2$) * 补货量($N$)
  = 补货量($N$) * [ (1-k) * 成本加成定价($F_1$) ]
 
$Q = F_1 * N_1 - 4.36 * N_2$

$-16\leq F_1 \leq 45.6$

$0\leq N_1 \leq 1.2$

$N_2 \geq 2.6758$

In [33]:
df.head()

,单品名,成本,亏损比,销量约束最小值,销量约束最大值,成本加成定价约束最小值,成本加成定价约束最大值,补货量最小值
0,白玉菇(袋),4.364556,6.57,0.8000,1.200,3.280,7.560,2.675800
1,菠菜,9.660000,18.51,1.7600,3.960,12.800,20.400,3.067861
2,菠菜(份),4.070000,9.43,1.6000,18.000,3.120,7.320,2.760296
3,菜心,4.620000,13.70,0.6576,4.944,5.264,7.932,2.896871
4,虫草花(份),2.600000,9.43,0.8000,4.800,3.600,5.640,2.760296


In [34]:
def find_best(i):
    score, score_F, score_N_buhuo, name = [], [], [], []

    # F(成本加成定价)的取值在df的第6和第7列之间
    for F in range(int(df.iloc[i, 5] * 10), int(df.iloc[i, 6] * 10 + 1)):
        F = F / 10.0

        # 如果销售约束最小值大于等于0
        if df.iloc[i, 3] >= 0:


            # N(销售量)的取值在df的第4列和第5列之间
            for N in range(int(df.iloc[i, 3] * 10), int(df.iloc[i, 4] * 10 + 1)):  
                N = N / 10.0  

                # 如果销量比最小展出量小，那补货量就是df的第八列（最小补货量）
                if N <= 2.5:
                    score.append(F * N - df.iloc[i, 1] * df.iloc[i, 7])
                    score_N_buhuo.append(df.iloc[i, 7])
                    score_F.append(F)
                    name.append(df.iloc[i, 0])
                # 如果销量比最小展出量大，那补货量就是 N / (1 - 损耗率)
                else:
                    score.append(F * N - df.iloc[i, 1] * (N / (1 - df.iloc[i, 2] / 100)))
                    score_N_buhuo.append(N / (1 - df.iloc[i, 2] / 100))
                    score_F.append(F)
                    name.append(df.iloc[i, 0])

        else:


            for N in range(0, int(df.iloc[i, 4] * 10 + 1)):  
                N = N / 10.0  

                # 如果销量比最小展出量小，那补货量就是df的第八列（最小补货量）
                if N <= 2.5:
                    score.append(F * N - df.iloc[i, 1] * df.iloc[i, 7])
                    score_N_buhuo.append(df.iloc[i, 7])
                    score_F.append(F)
                    name.append(df.iloc[i, 0])
                # 如果销量比最小展出量大，那补货量就是 N / (1 - 损耗率)
                else:
                    score.append(F * N - df.iloc[i, 1] * (N / (1 - df.iloc[i, 2] / 100)))
                    score_N_buhuo.append(N / (1 - df.iloc[i, 2] / 100))
                    score_F.append(F)
                    name.append(df.iloc[i, 0])


    max_score_index = score.index(max(score))
    max_F = score_F[max_score_index]
    max_N = score_N_buhuo[max_score_index]
    max_name = name[max_score_index]

    #print(name[i])
    #print("最大的score值:", max(score))
    #print("对应的F1值:", max_F)
    #print("对应的N1值:", max_N)

    return max(score), max_F, max_N, max_name

<br><br>

In [38]:
score, F_score, N_score, label = [], [], [], []
for i in range(0, len(df)):
    S, F, N, names = find_best(i)
    score.append(S)
    F_score.append(F)
    N_score.append(N)
    label.append(names)

白玉菇(袋)


菠菜
菠菜(份)
菜心
虫草花(份)
高瓜(1)
高瓜(2)
海鲜菇(包)
红椒(2)
红莲藕带
红薯尖
洪湖藕带
姜蒜小米椒组合装(小份)
金针菇(盒)
净藕(1)
菱角
螺丝椒
螺丝椒(份)
木耳菜
木耳菜(份)
奶白菜
七彩椒(2)
青红杭椒组合装(份)
青茄子(1)
青线椒(份)
上海青
双孢菇(盒)
娃娃菜
外地茼蒿
芜湖青椒(1)
西兰花
西峡花菇(1)
鲜木耳(份)
苋菜
小米椒(份)
小青菜(1)
小皱皮(份)
蟹味菇与白玉菇双拼(盒)
野生粉藕
圆茄子(2)
云南生菜
云南生菜(份)
云南油麦菜
云南油麦菜(份)
长线茄
枝江青梗散花
竹叶菜
紫茄子(1)
紫茄子(2)


In [39]:
df = pd.DataFrame({'菜名': label, '得分': score, '成本加成定价': F_score, '补货量': N_score})
df.head()

,菜名,得分,成本加成定价,补货量
0,白玉菇(袋),-2.678679,7.5,2.675800
1,菠菜,33.328561,20.4,4.785863
2,菠菜(份),50.512289,7.3,19.874131
3,菜心,12.478250,7.9,5.677868
4,虫草花(份),13.100603,5.6,5.299768


In [40]:
# 按照'Score'列的值从大到小排序
sorted_df = df.sort_values(by='得分', ascending=False)
sorted_df.head(33)

,菜名,得分,成本加成定价,补货量
30,西兰花,166.355927,16.6,22.261406
41,云南生菜(份),135.485234,5.8,76.846638
27,娃娃菜,116.008794,9.6,23.379820
14,净藕(1),94.439873,21.6,9.421978
43,云南油麦菜(份),86.246280,5.9,34.448493
48,紫茄子(2),83.692387,7.9,20.973065
29,芜湖青椒(1),76.886957,7.6,21.739130
44,长线茄,75.402632,14.7,11.278195
17,螺丝椒(份),72.123975,7.9,25.173899
11,洪湖藕带,70.801165,35.7,7.768269


In [41]:
column_sum = sorted_df['得分'].sum()
print("列的总和:", column_sum)

列的总和: 1749.8909083197689
